In [5]:
import sys
import pyfftw

import ROOT
from ROOT import gROOT
from ROOT import gStyle
%jsroot on

ROOT.gInterpreter.AddIncludePath('../../include/')
ROOT.gInterpreter.Declare('#include "RoEvent.h"')
ROOT.gSystem.Load('../../libs/libRoEvent.so')

gROOT.LoadMacro("../style/CMS/tdrstyle.C")
gROOT.ProcessLine("setTDRStyle();")
gROOT.LoadMacro("../style/CMS/CMS_lumi.C")

import pickle
from array import array
from pathlib import Path
import numpy as np
from scipy import signal
sys.path.append("../package/")
import lycro 

gStyle.SetOptFit(0)

card = "AMC ?"; run = 1177; thr = 70; pad = 30
pol = 1; N = 50; step = 20; percent = 0.03

#chs = range(0, 32); side = "Short"
#chs = range(32, 64); side = "Long"
chs = range(0, 64); side = "ALL"
chs_t = chs
dump = 0
elog = """\\url{https://elog.ip2i.in2p3.fr/wa105/Preparation+CB/108}"""
date = "June 2021"
place = "Lyon"
zoom_neg = -500; zoom_pos = 500
pick = "../../tmp/ju_r" + str(run) + "_thr-" + str(thr) + "_pad-" + str(pad) + ".pik"
root = "../../tmp/ro_r" + str(run) + ".root"
print("PICK file: {}".format(pick))
print("ROOT file: {}".format(root))
pdfpol = "Pos"
if(pol == -1):
    pdfpol = "Neg"
pathToSave = "../../tmp/r" + str(run) + "_" + side + "_" + pdfpol + "/fig/"
pdf = "../../tmp/r" + str(run) + "_" + side + "_" + pdfpol + "/viewer_r" + str(run) + "_" + side + "_" + pdfpol + ".tex"
print("PDF file: {}".format(pdf))

Path("../../tmp/r" + str(run) + "_" + side + "_" + pdfpol).mkdir(parents=True, exist_ok=True)
Path("../../tmp/r" + str(run) + "_" + side + "_" + pdfpol + "/fig").mkdir(parents=True, exist_ok=True)

# read ju_signals
ju_signals = []
with open(pick, "rb") as f:
    for _ in range(pickle.load(f)):
        ju_signals.append(pickle.load(f))  
print(len(ju_signals))

ped, _ = lycro.pedestal(ju_signals, pad = pad)
prof, prof_N = lycro.profile(ju_signals, pol = pol, pad = pad)

ped_aling, ped_avg = lycro.pedestal(ju_signals, pad = pad, align = 1)

gr_ped_avg, avgped = lycro.average(ped_avg)
prof_correct, correct_N = lycro.profile(ju_signals, pol = pol, pad = pad, align = 0, avgped = avgped)
prof_cut = lycro.cutProf(prof_correct, pol = pol, pad = pad)


chs_thr, min_thr, max_thr = lycro.filter(ju_signals, prof_cut, chs, pol = pol, N = N, step = step)
print(len(chs), chs)
print(len(chs_thr), chs_thr)
prof_final_N, prof_final, gr_h, gr_sum, h_sum, h_avg = lycro.profSumHeight(ju_signals, pol = pol, pad = pad, align = 0, chs = chs, chs_thr = chs_thr)


ptype = "profile"

name = "prof_final"; title = "run {} | {} | {} | {} | Profile of signals after filtering N={}".format(run, card, side, pdfpol, prof_final_N)
lycro.plot(prof_final, name, title, pathToSave, ptype, "tick", "ADCu", -2048, 2048, 0, 2*pad)
name = "prof_final_zoom"; title = "run {} | {} | {} | {} | (ZOOM) Profile of signals after filtering N={}".format(run, card, side, pdfpol, prof_final_N)
lycro.plot(prof_final, name, title, pathToSave, ptype, "tick", "ADCu", zoom_neg, zoom_pos, 0, 2*pad)

name = "profile"; title = "run {} | {} | {} | {} | Profile of all signals N={}".format(run, card, side, pdfpol, prof_N)
lycro.plot(prof, name, title, pathToSave, ptype, "tick", "ADCu",-2048, 2048, 0, 2*pad)
name = "profile_zoom"; title = "run {} | {} | {} | {} | (ZOOM) Profile of all signals N={}".format(run, card, side, pdfpol, prof_N)
lycro.plot(prof, name, title, pathToSave, ptype, "tick", "ADCu", zoom_neg, zoom_pos, 0, 2*pad)

name = "height"; title = "(height) run {} | {} | {} | {}".format(run, card, side, pdfpol)
lycro.plot(gr_h, name, title, pathToSave, "hsum", "chid", "height",1, -1, 1, -1)
name = "sum"; title = "(sum) run {} | {} | {} | {}".format(run, card, side, pdfpol)
lycro.plot(gr_sum, name, title, pathToSave, "hsum", "chid", "sum", 1, -1, 1, -1)
figs = []
fig_sum = []
fig_height = []
str_h = ""
str_sum = ""
if(dump):
    for sig in ju_signals:
        index = 0; index = chs.index(sig.ch)
        if(sig.unequal == 1 and sig.overlap < (min_thr[index] + (chs_thr[index] - min_thr[index])*percent)):
            ch = sig.ch
            ev = sig.ev
            name = "run" + str(run) + "_ch" + str(ch) + "_ev" + str(ev)
            cro = lycro.findCRO(root, ev, ch)

            d_cro = array('f'); d_tick = array('f'); err = array('f')
            for sa in range(len(cro)):
                d_cro.append(cro[sa])
                d_tick.append(sa)
                err.append(0)
            gr = ROOT.TGraphErrors(len(d_tick), d_tick, d_cro, err, err)
            gr.SetTitle("gr_" + name); gr.SetName("gr_" + name)
            gr.GetXaxis().SetTitle("tick"); gr.GetYaxis().SetTitle("ADCu")
            title = name + "        "
            lycro.plot(gr, name, title, pathToSave, "event", "tick", "ADCu", 0, 4096, 0, 10000)
            figs.append(name);

total_check = 0            
for sig in ju_signals:
        index = 0; index = chs.index(sig.ch)
        if(sig.unequal == 1):
            ch = sig.ch
            ev = sig.ev
            name = "run" + str(run) + "_ch" + str(ch) + "_ev" + str(ev)
            cro = lycro.findCRO(root, ev, ch)

            d_cro = array('f'); d_tick = array('f'); err = array('f')
            for sa in range(len(cro)):
                d_cro.append(cro[sa])
                d_tick.append(sa)
                err.append(0)
            gr = ROOT.TGraphErrors(len(d_tick), d_tick, d_cro, err, err)
            gr.SetTitle("gr_" + name); gr.SetName("gr_" + name)
            gr.GetXaxis().SetTitle("tick"); gr.GetYaxis().SetTitle("ADCu")
            title = name + "        "
            lycro.plot(gr, name, title, pathToSave, "event", "tick", "ADCu", 0, 4096, 0, 10000)
            figs.append(name); 
            total_check +=1
            if(total_check > (prof_N - prof_final_N)/100):
                break;

#######
for h in range(len(h_sum)):
    print(h_sum[h].GetName(), end =" ")
print("")
for h in range(len(h_sum)):
    lycro.plot(h_sum[h], h_sum[h].GetName(), "run {} | {} | {} | {} | {} | mean {:02f} | rms {:02f}".format(run, card, side, pdfpol, h_sum[h].GetName(), h_sum[h].GetMean(), h_sum[h].GetRMS()), pathToSave, "", "sum", "entries", 1, -1, 1, -1)  
    fig_sum.append(h_sum[h].GetName())
    #print(round(h_sum[h].GetMean(), 1), end = " ")
    str_sum += str(round(h_sum[h].GetMean(), 1))+ " " 
for h in range(len(h_avg)):
    print(h_avg[h].GetName(), end = " ")
print("")
for h in range(len(h_avg)):
    lycro.plot(h_avg[h], h_avg[h].GetName(), "run {} | {} | {} | {} | {} | mean {:02f} | rms {:02f}".format(run, card, side, pdfpol, h_avg[h].GetName(), h_avg[h].GetMean(), h_avg[h].GetRMS()), pathToSave, "", "height", "entries", 1, -1, 1, -1)  
    fig_height.append(h_avg[h].GetName())
    #print(round(h_avg[h].GetMean(), 1), end = " ")
    str_h += str(round(h_avg[h].GetMean(),1)) + " " 
print("")
#######
name = "profile_cut"; title = "Cut".format(prof_N)
lycro.plot(prof_cut, name, title, pathToSave, ptype, "tick", "ADCu",-2048, 2048, 0, 2*pad)
name = "profile_cut_zoom"; title = "(ZOOM) Cut".format(prof_N)
lycro.plot(prof_cut, name, title, pathToSave, ptype, "tick", "ADCu", zoom_neg, zoom_pos, 0, 2*pad)

name = "mainProf"; title = "#bf{Profile of all signals}"
lycro.plot(prof_correct, name, title, pathToSave, ptype, "tick", "ADCu", -2048, 2048, 0, 2*pad)
name = "mainProf_zoom"; title = "#bf{Profile of all signals (ZOOM)}"
lycro.plot(prof_correct, name, title, pathToSave, ptype, "tick", "ADCu", -200, 200, 0, 2*pad)

name = "pedestal"; title = "#bf{Profile of all pedestal}"
lycro.plot(ped, name, title, pathToSave, ptype, "tick", "ADCu", -2048, 2048, 0, 2*pad)
name = "pedestal_zoom"; title = "#bf{Profile of all pedestal (ZOOM)}"
lycro.plot(ped, name, title, pathToSave, ptype, "tick", "ADCu", -50, 50, 0, 2*pad)

name = "pedestal_aling"; title = "#bf{Profile of all pedestal Aling}"
lycro.plot(ped_aling, name, title, pathToSave, ptype, "tick", "ADCu", -2048, 2048, 0, 2*pad)
name = "pedestal_aling_zoom"; title = "#(ZOOM) bf{Profile of all pedestal Aling}"
lycro.plot(ped_aling, name, title, pathToSave, ptype, "tick", "ADCu", -50, 50, 0, 2*pad)

def figure(path2First, path2Second, caption, label):
    output = "\\begin{figure}[h!]\n"
    output += "\\begin{minipage}[h]{0.47\linewidth}\n"
    
    output += "\\center{\includegraphics[width=1\linewidth]{"
    output += path2First
    output += "}} \n"
    
    output += "\\end{minipage} \n"
    output += "\\hfill \n"
    output += "\\begin{minipage}[h]{0.47\linewidth} \n"
    
    output += "\\center{\includegraphics[width=1\linewidth]{"
    output += path2Second
    output += "}} \n"
    
    output += "\\end{minipage} \n"
    
    output += "\\caption{"
    output += caption
    output += "} \n"
    
    output += "\\label{"
    output += label
    output += "} \n"
    
    output += "\\end{figure} \n"
    output += "\\FloatBarrier \n"
    
    return output

def event_figure(path):
    output = "\\begin{figure}[!ht]\n"
    
    output += "\\center{\includegraphics[width=0.9\linewidth]{"
    output += path
    output += "}} \n" 
    
    output += "\\end{figure} \n"
    output += "\\FloatBarrier \n"
    
    return output


with open(pdf,'w') as file:
    file.write('\\documentclass[12pt, a4paper]{article}\n')
    file.write('\\usepackage[utf8]{inputenc}\n')
    file.write('\\usepackage[english]{babel}\n')
    file.write('\\usepackage[T1]{fontenc}\n')
    file.write('\\usepackage{graphicx}\n')
    file.write('\\usepackage{geometry}\n')
    file.write('\\usepackage{color,soul}\n')
    file.write('\\DeclareRobustCommand{\hlcyan}[1]{{\sethlcolor{cyan}\hl{#1}}}\n')
    file.write('\\usepackage[document]{ragged2e}\n')
    file.write('\\usepackage{array}\n')
    file.write('\\usepackage{graphicx}\n')
    file.write('\\usepackage{placeins}\n')
    file.write('\\usepackage{amssymb}\n')
    file.write('\\usepackage{wrapfig}\n')
    file.write('\\usepackage{hyperref}\n')
    file.write('\\usepackage{listings}\n')
    
    file.write('\\newcommand{\+}[1]{\ensuremath{\mathbf{#1}}}\n')
    file.write('\\geometry{top=2.5cm, bottom=2.5cm, right=2.5cm, left=2.5cm}\n')
    percent
    file.write('\\begin{document}\n')
    file.write('\\noindent Institut de Physique des 2 Infinis de Lyon \hfill Date, Place \\\\ \n')
    file.write('\\noindent Shchablo Konstantin (shchablo@gmail.com) \hfill ' + date + ', ' + place  + ' \\\\ \n')
    file.write('\\center\n')
    namepol = "Positive"
    if(pol == -1):
        namepol = "Negative"
    file.write('\\section*{Signal View: Run-' + str(run) + " " + card  +  " " + side + " " + namepol + '}\n')
    file.write('ELOG: ' + elog +'\n')
    
    file.write('\\justify\n')
    #file.write('The processing includes filtering based on the selection of the signals that correspond to the most common pattern. The pedestal corrected with a spectrum subtraction algorithm.  The height found with help of the resampling method. The sum calculated as the integral of the signal in the given window.\n')
    file.write('\\\\ \\\\ GitHub: (https://github.com/shchablo/CRO)\n')   
    
    file.write('\\tableofcontents\n')
    
    file.write('\\newpage\n')
    #file.write('\\newcommand{\+}[1]{\ensuremath{\mathbf{#1}}}\n')
    file.write('\\section{Results: Profile; Filtered Profile; Height; Sum.}\n') 
    file.write(figure( "./fig/height.pdf", "./fig/sum.pdf", "height:" + " sum:", "hsum"))
    file.write(figure( "./fig/prof_final.pdf", "./fig/prof_final_zoom.pdf", "File: Prof Final.", "prof_final"))
    file.write(figure( "./fig/profile.pdf", "./fig/profile_zoom.pdf", "File: Profile.", "profile"))
    
    file.write('\\newpage\n')
    file.write('\\section{Pedestal; Cut Profile; }\n')
    file.write(figure( "./fig/profile_cut.pdf", "./fig/profile_cut_zoom.pdf", "File: Profile Cut.", "profile_cut"))
    file.write(figure( "./fig/pedestal.pdf", "./fig/pedestal_zoom.pdf", "File: Pedestal.", "pedestal"))
    file.write(figure( "./fig/pedestal_aling.pdf", "./fig/pedestal_aling_zoom.pdf", "File: Pedestal Aling.", "paling"))
    
    file.write('\\newpage\n')
    file.write('\\section{Height.}\n')
    
    for f in fig_height:
        file.write(event_figure("./fig/" + f + ".pdf"))
    
    file.write('\\section{Integral.}\n')
    
    for f in fig_sum:
        file.write(event_figure("./fig/" + f + ".pdf"))
    
    value = prof_N - prof_final_N
    file.write('\\section{Examples of some unusually signals (In total ' + str(value)  + '). }\n')
    for f in figs:
        file.write(event_figure("./fig/" + f + ".pdf"))
    
    file.write('\\end{document}\n')

    
print("DONE")



PICK file: ../../tmp/ju_r1177_thr-70_pad-30.pik
ROOT file: ../../tmp/ro_r1177.root
PDF file: ../../tmp/r1177_ALL_Pos/viewer_r1177_ALL_Pos.tex
124238
64 range(0, 64)
64 [31894, 33296, 32085, 31991, 32108, 32737, 31468, 32161, 33478, 33138, 32558, 33271, 33159, 32977, 32107, 34132, 33073, 32861, 32952, 32315, 32213, 33214, 33566, 33638, 1080, 973, 30643, 30486, 33396, 32134, 32341, 32822, 32229, 32082, 32709, 33204, 32564, 32553, 31489, 31498, 32961, 33420, 33959, 33306, 33680, 32711, 32609, 32094, 32660, 31947, 32530, 33270, 32907, 33606, 32339, 32548, 33902, 32529, 31352, 31543, 33600, 32656, 33157, 33026]
sum_ch0 sum_ch1 sum_ch2 sum_ch3 sum_ch4 sum_ch5 sum_ch6 sum_ch7 sum_ch8 sum_ch9 sum_ch10 sum_ch11 sum_ch12 sum_ch13 sum_ch14 sum_ch15 sum_ch16 sum_ch17 sum_ch18 sum_ch19 sum_ch20 sum_ch21 sum_ch22 sum_ch23 sum_ch24 sum_ch25 sum_ch26 sum_ch27 sum_ch28 sum_ch29 sum_ch30 sum_ch31 sum_ch32 sum_ch33 sum_ch34 sum_ch35 sum_ch36 sum_ch37 sum_ch38 sum_ch39 sum_ch40 sum_ch41 sum_ch42 sum_ch43 

Warning in <TROOT::Append>: Replacing existing TH1: pedesta (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: profile_pos (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: pedesta (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: profile_pos (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: profile_cut_pos (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sum_ch0 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: height_ch0 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sum_ch1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: height_ch1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sum_ch2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: height_ch2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing ex

In [6]:
print("SUM")
print(str_sum)

SUM
627.6 641.2 614.5 640.7 623.4 635.9 588.2 636.0 676.0 665.9 650.5 650.1 649.9 647.2 640.0 673.5 652.2 657.9 652.2 654.2 622.1 650.5 667.7 675.4 891.8 758.3 570.3 575.3 629.6 615.7 671.5 649.5 637.4 634.3 637.7 652.4 614.4 653.1 608.8 634.4 657.5 654.3 676.4 648.3 658.0 651.3 652.1 649.8 663.7 655.2 663.7 663.4 672.9 667.2 655.2 671.2 640.6 613.5 583.0 589.1 637.9 668.5 640.5 657.0 


In [7]:
print("height")
print(str_h)

height
109.2 107.1 105.9 110.0 105.1 106.5 100.6 104.7 114.5 113.7 110.9 112.5 110.6 109.4 109.9 114.3 113.8 112.6 111.9 115.0 110.5 115.0 114.6 115.1 105.9 92.0 99.1 96.4 107.6 106.9 114.3 111.1 107.9 107.2 109.3 109.4 103.8 110.0 102.3 105.6 112.4 114.1 112.8 110.7 113.1 112.7 111.8 112.7 114.3 113.3 113.5 115.0 115.7 115.3 115.3 114.3 108.2 103.6 101.8 101.3 108.4 112.5 109.8 112.0 
